# 📌 Grocery Customer Churn (Abandono de Clientes de Supermercado)

## Acerca del conjunto de datos

Este conjunto de datos contiene información detallada sobre las transacciones, el comportamiento y los datos demográficos de los clientes de una tienda de comestibles. Incluye información de los clientes, datos transaccionales y métricas de comportamiento, lo que lo hace ideal para crear modelos predictivos para la pérdida de clientes y el análisis del valor de vida del cliente (CLV).

La columna objetivo para la predicción de abandono es la tasa de abandono, que indica si un cliente se ha dado de baja (1) o ha permanecido activo (0). El conjunto de datos está diseñado para crear modelos y predecir la retención de clientes, analizar su comportamiento y pronosticar su valor de vida.

Características principales:

* Información del cliente: incluye detalles demográficos como edad, sexo, nivel de ingresos, estado civil, nivel educativo y ocupación.

* Datos transaccionales: contiene información sobre cada transacción, como la fecha de la transacción, la cantidad, el precio, la categoría del producto y el método de pago.

* Métricas de comportamiento del cliente: incluye características como el valor de compra promedio, la frecuencia de compra y la fecha de la última compra.

* Datos promocionales: incluye detalles sobre las promociones, incluido el tipo, la eficacia y el público objetivo.

* Churn: La columna de destino indica si el cliente se ha dado de baja (1) o ha permanecido activo (0).

* Tamaño del conjunto de datos: este conjunto de datos contiene 35.843 filas y 26 atributos (incluida la columna de abandono).

Este estudio incluye las tareas del Data Engineer:

* Obtención y tratamiento de datos a través de una API externa.

* Implementación del proceso ETL (Extracción, Transformación y Carga) para la depuración y organización de la información.

* Desarrollo de visualizaciones clave para detectar patrones y comportamientos relevantes.

* Exploración de los datos (EDA técnico) y elaboración de un reporte con hallazgos significativos, para posterior EDA profundo.

##  Extracción

In [692]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')
from google.colab import files

# Abrir un cuadro para subir archivos
uploaded = files.upload()

for nombre_archivo in uploaded.keys():
    df = pd.read_csv(nombre_archivo)


Saving Grocery_Customer_Churn_Data.csv to Grocery_Customer_Churn_Data.csv


## Inspección general del dataset

In [693]:
# Configuraciones de pandas

pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", "{:.2f}".format)


In [694]:
# Mostrar las filas del DataFrame

df.head()
df.tail()



,customer_id,age,gender,income_bracket,loyalty_program,membership_years,marital_status,number_of_children,education_level,occupation,transaction_id,transaction_date,product_category,quantity,unit_price,avg_purchase_value,purchase_frequency,last_purchase_date,avg_discount_used,online_purchases,in_store_purchases,total_sales,total_transactions,total_items_purchased,promotion_type,promotion_effectiveness,days_since_last_purchase,churn
35839,C1499,20,Other,Low,Yes,1,Single,1,High School,Employed,TX-f28be738,2024-05-30,Beauty,2,58.05,116.10,Low,2024-05-30,0.21,2,0,91.72,50,63,NaN,High,578,0
35840,C1499,20,Other,Low,Yes,1,Single,1,High School,Employed,TX-cd6de9d7,2024-02-13,Beauty,3,36.03,108.09,NaN,2024-02-13,0.13,0,3,94.04,50,64,Discount,Low,685,0
35841,C1499,20,Other,Low,Yes,1,Single,1,High School,Employed,TX-ca3f9836,2024-04-11,Sports,3,58.00,174.00,High,2024-04-11,0.28,3,0,NaN,50,64,BOGO,Low,627,0
35842,C1499,20,Other,Low,Yes,1,Single,1,High School,Employed,TX-63255845,2024-08-21,Sports,4,18.49,73.96,Medium,2024-08-21,0.12,4,0,65.08,50,65,BOGO,High,495,0
35843,C1499,20,Other,Low,Yes,1,Single,1,High School,Employed,TX-1c7b49d1,2024-04-22,Books,3,72.15,216.45,Medium,2024-04-22,0.24,3,0,164.50,50,64,NaN,Low,616,0


In [695]:
# Atributo del DataFrame
df.shape

(35844, 28)

In [696]:
# Obtener información general del DataFrame

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35844 entries, 0 to 35843
Data columns (total 28 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   customer_id               35844 non-null  object 
 1   age                       35844 non-null  int64  
 2   gender                    35844 non-null  object 
 3   income_bracket            35844 non-null  object 
 4   loyalty_program           35844 non-null  object 
 5   membership_years          35844 non-null  int64  
 6   marital_status            35844 non-null  object 
 7   number_of_children        35844 non-null  int64  
 8   education_level           35844 non-null  object 
 9   occupation                35844 non-null  object 
 10  transaction_id            35844 non-null  object 
 11  transaction_date          35844 non-null  object 
 12  product_category          35844 non-null  object 
 13  quantity                  35844 non-null  int64  
 14  unit_p

**Nota:**

El dataset contiene 35.843 registros y 28 columnas, combinando información de clientes y transacciones.

Se observa una mezcla de variables:

* Identificadores (customer_id, transaction_id)

* Variables demográficas del cliente

* Variables transaccionales

* Variables agregadas históricas

Esto indica que el dataset no está normalizado y presenta múltiples niveles de granularidad.

* transaction_date, last_purchase_date → object ❌

* purchase_frequency → debería ser categórica (object está bien, pero validar valores)

* promotion_effectiveness → categórica, no numérica

## Clasificar variables

In [697]:
num_cols = [
    "age", "membership_years", "number_of_children",
    "quantity", "unit_price", "avg_purchase_value",
    "purchase_frequency", "avg_discount_used",
    "online_purchases", "in_store_purchases",
    "total_sales", "total_transactions",
    "total_items_purchased", "promotion_effectiveness",
    "days_since_last_purchase"
]

categorical_cols = [
    "gender", "income_bracket", "loyalty_program",
    "marital_status", "education_level", "occupation",
    "product_category", "promotion_type", "churn"
]

date_cols = [
    "transaction_date", "last_purchase_date"
]

id_cols = [
    "customer_id", "transaction_id"
]


Se realizó una clasificación explícita de columnas por tipo lógico (numéricas, categóricas, fechas e identificadores) con el objetivo de:

* Facilitar validaciones de calidad de datos

* Aplicar transformaciones específicas por tipo

* Mejorar la mantenibilidad del pipeline ETL

## Detección de valores nulos

In [698]:
nulls = df.isna().sum().sort_values(ascending=False)
nulls[nulls > 0]


,0
promotion_type,14241
total_sales,3614
avg_purchase_value,3531
purchase_frequency,1779


**Nota:**

Se identifican valores faltantes principalmente en variables relacionadas con promociones, frecuencia de compra y métricas agregadas.

Columnas con mayor cantidad de nulos:

* promotion_type: 14241
* avg_purchase_value: 3531
* purchase_frequency: 1779
* total_sales: 3614

Esto sugiere que:

* No todas las transacciones estuvieron asociadas a promociones

* Algunas métricas agregadas pueden no haber sido calculadas para todos los registros

Estos nulos no se consideran errores automáticamente, sino que requieren una decisión de negocio (imputación, categorización explícita o exclusión según el caso).

* promotion_type nulo → no todas las transacciones tuvieron promoción

* purchase_frequency y avg_purchase_value → no estén calculadas para todos los clientes

* total_sales nulo → inconsistencia, posiblemente falte para clientes sin historial consolidado


## Duplicados y unicidad

In [699]:
df["customer_id"].duplicated().sum()


np.int64(35344)

In [700]:
df["transaction_id"].duplicated().sum()


np.int64(0)

In [701]:
df.groupby("customer_id")["transaction_id"].nunique().describe()


,transaction_id
count,500.00
mean,71.69
std,0.46
min,71.00
25%,71.00
50%,72.00
75%,72.00
max,72.00


In [702]:
df["transaction_id"].nunique()

35844

In [703]:
df["customer_id"].value_counts().head()

,count
customer_id,
C1343,72
C1342,72
C1341,72
C1340,72
C1339,72


**Nota:**

customer_id duplicados: 35.343

transaction_id:

* count = 500

* mean = 71.69

* std = 0.46

Conclusión estructural:

* Cada transaction_id aparece ~72 veces

* Pero cada customer_id aparece muchas veces

* transaction_id NO es una transacción. Además  transaction_id identifica un CUSTOMER, no un evento




## EDA técnico

### Variables Númericas y Fechas

In [704]:
df[num_cols ].describe()

,age,membership_years,number_of_children,quantity,unit_price,avg_purchase_value,avg_discount_used,online_purchases,in_store_purchases,total_sales,total_transactions,total_items_purchased,days_since_last_purchase
count,35844.00,35844.00,35844.00,35844.00,35844.00,32313.00,35844.00,35844.00,35844.00,32230.00,35844.00,35844.00,35844.00
mean,44.22,7.68,2.07,3.00,50.84,149.21,0.29,2.14,0.96,163.16,29.61,58.21,831.30
std,14.94,4.40,1.42,1.41,21.70,95.65,1.21,1.86,1.67,357.30,13.42,26.39,207.37
min,18.00,1.00,0.00,1.00,0.02,-1.77,0.00,0.00,0.00,-896.23,5.00,10.00,-318.00
25%,32.00,4.00,1.00,2.00,36.54,77.71,0.08,0.00,0.00,66.36,19.00,34.00,706.00
50%,45.00,8.00,2.00,3.00,50.03,130.92,0.15,2.00,0.00,112.72,30.00,60.00,843.00
75%,57.00,11.00,3.00,4.00,63.94,199.74,0.23,3.00,2.00,173.45,42.00,80.00,971.00
max,70.00,15.00,4.00,12.00,199.75,909.10,14.96,12.00,11.00,6233.02,51.00,109.00,1273.00


In [705]:
# Lista de columnas numéricas a evaluar

# Función para detectar outliers por regla de negocio
def detect_anomalies(df, cols):
    anomalies = {}

    for col in cols:
        if col not in df.columns:
            continue

        # Forzar conversión numérica
        series_raw = df[col]
        series = pd.to_numeric(series_raw, errors="coerce")

        n_missing = series.isnull().sum()
        n_negative = (series < 0).sum()
        n_zero = (series == 0).sum()

        min_val = series.min()
        max_val = series.max()
        mean_val = series.mean()
        std_val = series.std()

        if pd.notna(std_val) and std_val > 0:
            lower_limit = mean_val - 3 * std_val
            upper_limit = mean_val + 3 * std_val
            n_outliers = ((series < lower_limit) | (series > upper_limit)).sum()
        else:
            n_outliers = 0

        anomalies[col] = {
            "missing": int(n_missing),
            "negative": int(n_negative),
            "zero": int(n_zero),
            "min": min_val,
            "max": max_val,
            "mean": mean_val,
            "std": std_val,
            "outliers_3sigma": int(n_outliers)
        }

    return pd.DataFrame(anomalies).T


In [706]:
# Evaluación numéricas
num_anomalies = detect_anomalies(df, num_cols)
display(num_anomalies.sort_values(by="outliers_3sigma", ascending=False))

# Fechas: detectar fechas futuras o inconsistentes
today = pd.Timestamp.today()

date_cols = ["transaction_date", "last_purchase_date"]

for col in date_cols:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors="coerce")


for col in date_cols:
    if col in df.columns:
        n_missing = df[col].isnull().sum()
        n_future = (df[col] > today).sum()
        min_date = df[col].min()
        max_date = df[col].max()

        print(
            f"{col}: "
            f"missing={n_missing}, "
            f"future_dates={n_future}, "
            f"min={min_date}, "
            f"max={max_date}"
        )


,missing,negative,zero,min,max,mean,std,outliers_3sigma
in_store_purchases,0.00,0.00,24812.00,0.00,11.00,0.96,1.67,732.00
avg_discount_used,0.00,0.00,564.00,0.00,14.96,0.29,1.21,500.00
days_since_last_purchase,0.00,318.00,1.00,-318.00,1273.00,831.30,207.37,500.00
avg_purchase_value,3531.00,2.00,0.00,-1.77,909.10,149.21,95.65,394.00
total_sales,3614.00,18.00,0.00,-896.23,6233.02,163.16,357.30,359.00
online_purchases,0.00,0.00,10626.00,0.00,12.00,2.14,1.86,247.00
unit_price,0.00,0.00,0.00,0.02,199.75,50.84,21.70,236.00
quantity,0.00,0.00,0.00,1.00,12.00,3.00,1.41,166.00
age,0.00,0.00,0.00,18.00,70.00,44.22,14.94,0.00
membership_years,0.00,0.00,0.00,1.00,15.00,7.68,4.40,0.00


transaction_date: missing=0, future_dates=0, min=2022-07-05 00:00:00, max=2024-11-10 00:00:00
last_purchase_date: missing=0, future_dates=0, min=2022-07-05 00:00:00, max=2024-11-10 00:00:00


**Nota:**

1️⃣ Columnas numéricas

* age, membership_years, number_of_children, quantity, unit_price

Sin valores faltantes

Sin valores negativos

Sin outliers según criterio 3σ
✅ Columnas saneadas y confiables

* avg_purchase_value

3.531 valores nulos → ya identificado previamente

2 valores negativos

Mínimo = -1.77 → valores pequeños, probablemente errores de transacción o ajustes mal calculados
⚠️ Requiere limpieza puntual (filtrar o imputar)

* purchase_frequency

35.844 valores nulos (100%)

➡️ Debe recalcularse

* total_sales

3.614 valores nulos → ya identificado previamente

18 valores negativos

Mínimo = -896.23 → errores claros de cálculo o registros de reverso mal tratados
⚠️ Revisar fórmula

* online_purchases, in_store_purchases
Alta cantidad de ceros

online_purchases: 10.626 ceros

in_store_purchases: 24.812 ceros
ℹ️ Puede ser consistente con el comportamiento del cliente (canales excluyentes)
➡️ Validar contra reglas de negocio

* promotion_effectiveness

35.844 valores nulos (100%)
❌ Variable inutilizable en su estado actual
➡️ Recalcular o eliminar

* days_since_last_purchase

318 valores negativos

Mínimo = -318
❌ Error sistémico confirmado
➡️ El problema no es la fecha en sí, sino el cálculo

2️⃣ Columnas de fecha

* transaction_date y last_purchase_date

No hay nulos, no hay fechas futuras → ✅ consistentes


3️⃣ Conclusiones parciales

Las fechas son confiables, no hay que imputar ni corregir transaction_date ni last_purchase_date.

days_since_last_purchase debe recalcularse para eliminar los negativos.

purchase_frequency y promotion_effectiveness no se pueden usar tal cual, requieren recalculo o imputación.

avg_purchase_value y total_sales tienen algunos valores negativos. Revisar cómo se calcularon (probablemente quantity * unit_price para total_sales).

El resto de variables numéricas están limpias y listas para análisis o derivación de features.

In [707]:
(df["age"] < 0).sum(), (df["age"] > 100).sum()


(np.int64(0), np.int64(0))

In [708]:
(df["unit_price"] < 0).sum()


np.int64(0)

In [709]:
(df["quantity"] <= 0).sum()


np.int64(0)

In [710]:
(df["transaction_date"] > pd.Timestamp.today()).sum()


np.int64(0)

In [711]:
# Detección de anomalías lógicas

anomalies = {}

# Ventas negativas
anomalies["negative_total_sales"] = df[df["total_sales"] < 0].shape[0]

# Cantidades inválidas
anomalies["invalid_quantity"] = df[df["quantity"] <= 0].shape[0]

# Precios inválidos
anomalies["invalid_unit_price"] = df[df["unit_price"] <= 0].shape[0]

# Edades fuera de rango lógico
anomalies["invalid_age"] = df[(df["age"] < 18) | (df["age"] > 100)].shape[0]

# Métricas temporales inconsistentes
if "days_since_last_purchase" in df.columns:
    anomalies["negative_days_since_last_purchase"] = (
        df[df["days_since_last_purchase"] < 0].shape[0]
    )

# Resumen de anomalías
anomaly_df = pd.DataFrame.from_dict(
    anomalies, orient="index", columns=["count"]
)

print("\nResumen de anomalías detectadas:")
display(anomaly_df)


Resumen de anomalías detectadas:


,count
negative_total_sales,18
invalid_quantity,0
invalid_unit_price,0
invalid_age,0
negative_days_since_last_purchase,318


**Nota:**

El análisis de calidad de datos identificó dos tipos principales de anomalías:

* Ventas totales negativas (negative_total_sales):
Se detectaron 18 registros con valores negativos, lo que puede estar asociado a devoluciones, cancelaciones o ajustes contables. Estos casos requieren validación de negocio para decidir si deben mantenerse, corregirse o excluirse.

* Días negativos desde la última compra (negative_days_since_last_purchase):
Se identificaron 318 registros con valores negativos, lo que indica inconsistencias temporales (por ejemplo, fechas de referencia mal definidas o eventos posteriores al “snapshot” de análisis). Esta anomalía sugiere un problema de alineación temporal más que de valores numéricos.

* No se detectaron anomalías en:

1.Cantidades (invalid_quantity)

2.Precios unitarios (invalid_unit_price)

3.Edad de clientes (invalid_age)

### Variables Catégoricas

In [712]:
for col in categorical_cols:
    print("\n", col)
    display(df[col].value_counts(dropna=False).head(10))



 gender


,count
gender,
Male,12116
Other,12113
Female,11615



 income_bracket


,count
income_bracket,
High,13769
Low,11747
Medium,10328



 loyalty_program


...

,count
product_category,
Electronics,6020
Home,5969
Books,5949
Sports,5914
Clothing,5856
Beauty,5838
Home Goods,112
Toys,102
Groceries,84



 promotion_type


,count
promotion_type,
NaN,14241
Discount,10617
BOGO,10566
Seasonal Discount,151
20% Off,136
Buy One Get One Free,133



 churn


,count
churn,
0,35700
1,144


**Nota:**

1️⃣ Género

Male: 12.116

Other: 12.112

Female: 11.615

✅ Hay una distribución bastante equilibrada entre géneros, con una ligera ventaja de “Male” y “Other” sobre “Female”. Esto es útil, porque no hay un sesgo fuerte hacia un género en los análisis posteriores.

2️⃣ Income Bracket (Nivel de ingreso)

High: 13.769

Low: 11.747

Medium: 10.328

✅ Predomina el nivel High, seguido de Low y Medium. Esto indica que la base de clientes tiene una proporción relativamente alta de ingresos altos, lo que puede influir en patrones de consumo y promociones.

3️⃣ Loyalty Program

Yes: 18.845

No: 16.999

✅ La mayoría de los clientes participa en un programa de fidelidad, pero hay una proporción significativa que no lo hace. Esto es clave para análisis de retención o churn.

4️⃣ Marital Status

Single: 13.182

Divorced: 11.333

Married: 11.329

✅ Hay más clientes solteros, aunque los divorciados y casados están bastante equilibrados. Esto puede impactar en patrones de compra según la etapa de vida.

5️⃣ Education Level

Master’s: 9.826

PhD: 9.108

Bachelor’s: 8.810

High School: 8.100

✅ La mayoría tiene educación avanzada, con predominio de Master’s y PhD. Esto podría correlacionarse con ingresos altos y ciertos hábitos de consumo.

6️⃣ Occupation

Employed: 7.812

Self-Employed: 7.750

Unemployed: 7.239

Retired: 6.880

Student: 6.163

✅ Hay una diversidad de ocupaciones, siendo la mayoría empleados o autónomos. Esto puede afectar frecuencia de compra y preferencias de productos.

7️⃣ Product Category

Electronics: 6.020

Home: 5.969

Books: 5.949

Sports: 5.914

Clothing: 5.856

Beauty: 5.838

Home Goods: 112

Toys: 102

Groceries: 84

✅ Las categorías más frecuentes son Electronics, Home, Books y Sports. Groceries, Toys y Home Goods son muy poco frecuentes, indicando enfoque en bienes duraderos o consumo discrecional.

8️⃣ Promotion Type

NaN: 14.241

Discount: 10.617

BOGO: 10.566

Seasonal Discount: 151

20% Off: 136

Buy One Get One Free: 133

✅ La mayoría de las transacciones no tienen promoción registrada (≈40%), pero cuando hay, predominan Discount y BOGO. Las promociones siguen siendo un factor relevante para analizar su efecto en ventas y churn

9️⃣ Churn

0 (No churn): 35.700

1 (Churn): 144

✅ La tasa de churn es extremadamente baja (<0,5%), lo que indica que la mayoría de los clientes permanecen activos.



Conclusión general

* La base de clientes es bastante equilibrada en género y estado civil, aunque ligeramente inclinada hacia solteros y hombres/other.

* Hay predominio de ingresos altos y educación avanzada, lo que podría afectar patrones de gasto.

* La participación en programas de fidelidad es alta, lo que permite analizar retención y efectividad de promociones.

* Los productos más populares son Electronics, Home, Books y Sports, y la mayoría de las compras incluyen alguna promoción.

* Los datos son en general balanceados, pero hay algunos valores faltantes en promotion_type, avg_purchase_value y otras variables que deberíamos considerar antes de modelar.

## Limpieza numérica y nulos

In [713]:
df_clean = df.copy()

In [714]:
# Columnas binarias que queremos inspeccionar
binary_cols = ["loyalty_program", "churn", "promo_flag"]

for col in binary_cols:
    if col in df_clean.columns:
        print(f"Valores únicos en '{col}':")
        print(df_clean[col].value_counts(dropna=False))
        print("-" * 40)


Valores únicos en 'loyalty_program':
loyalty_program
Yes    18845
No     16999
Name: count, dtype: int64
----------------------------------------
Valores únicos en 'churn':
churn
0    35700
1      144
Name: count, dtype: int64
----------------------------------------


In [715]:
df_clean = df.copy()

df_clean["gender"] = df_clean["gender"].fillna("unknown")

df_clean["age"] = df_clean["age"].clip(0, 100)
df_clean["age"] = df_clean["age"].fillna(df_clean["age"].median())

df_clean["unit_price"] = df_clean["unit_price"].clip(lower=0)


In [716]:
# Normalizar nombres de columnas
def clean_column_names(df_clean):
    df_clean = df.copy()
    df_clean.columns = (
        df.columns
        .str.strip()                # Quitar espacios al inicio/final
        .str.lower()                # Pasar todo a minúsculas
        .str.replace(" ", "_")      # Reemplazar espacios por _
        .str.replace("-", "_")      # Reemplazar guiones por _
        .str.replace(r"[^\w_]", "", regex=True)  # Quitar caracteres especiales
    )
    return df_clean

## Convertir fechas

In [717]:
# Convertir fechas
date_cols = ["transaction_date", "last_purchase_date"]
for col in date_cols:
    df_clean[col] = pd.to_datetime(df_clean[col], errors="coerce")

today = pd.Timestamp.today().normalize()

# Detectar y reportar fechas inconsistentes
for col in date_cols:
    n_missing = df[col].isnull().sum()
    n_future = (df[col] > today).sum()
    print(f"{col}: missing={n_missing}, future_dates={n_future}")

# Recalcular métrica correctamente
df_clean["days_since_last_purchase"] = (
    today - df_clean["last_purchase_date"]
).dt.days

# Seguridad final
df_clean.loc[
    df_clean["days_since_last_purchase"] < 0,
    "days_since_last_purchase"
] = 0

transaction_date: missing=0, future_dates=0
last_purchase_date: missing=0, future_dates=0


## Normalización de categorías y reducción de cardinalidad

In [718]:
for col in categorical_cols:

    # Reducir categorías raras (<1% frecuencia)
    freq = df_clean[col].value_counts(normalize=True)
    rare_labels = freq[freq < 0.01].index
    df_clean[col] = df_clean[col].replace(rare_labels, "other")

# Promotion type: rellenar NaN
df_clean["promotion_type"] = df_clean["promotion_type"].fillna("no_promotion")


In [719]:
df_clean["loyalty_program"] = df_clean["loyalty_program"].astype(str).str.strip().str.lower()

# Mapear Yes/No a 1/0
df_clean["loyalty_program"] = df_clean["loyalty_program"].map({"yes": 1, "no": 0})


## Métricas derivadas

In [720]:
df_clean["total_sales"] = df_clean["quantity"] * df_clean["unit_price"]
df_clean["avg_purchase_value"] = df_clean.groupby("customer_id")["total_sales"].transform("mean")
df_clean["purchase_frequency"] = df_clean.groupby("customer_id")["transaction_id"].transform("count") / df_clean["membership_years"].replace(0,1)

# Promotion effectiveness
df_clean["promo_flag"] = df_clean["promotion_type"].apply(lambda x: 0 if x=="no_promotion" else 1)
df_clean["promotion_effectiveness"] = df_clean.groupby("customer_id")["promo_flag"].transform("sum") / df_clean.groupby("customer_id")["transaction_id"].transform("count")

# Online vs in-store ratio
df_clean["total_purchases"] = df_clean["online_purchases"] + df_clean["in_store_purchases"]
df_clean["online_ratio"] = df_clean["online_purchases"] / df_clean["total_purchases"].replace(0,1)


In [721]:
# Mapear churn correctamente
df_clean['churn'] = df_clean['churn'].replace({'other': 1})

# Convertir a int
df_clean['churn'] = df_clean['churn'].astype(int)

# Verificar
print(df_clean['churn'].value_counts())

churn
0    35700
1      144
Name: count, dtype: int64


## Validación post-transformación

In [722]:
for col in num_cols:
    if col in df_clean.columns:
        print(f"{col}: min={df_clean[col].min()}, max={df_clean[col].max()}, mean={df_clean[col].mean():.2f}, nulos={df_clean[col].isnull().sum()}")

age: min=18, max=70, mean=44.22, nulos=0
membership_years: min=1, max=15, mean=7.68, nulos=0
number_of_children: min=0, max=4, mean=2.07, nulos=0
quantity: min=1, max=12, mean=3.00, nulos=0
unit_price: min=0.02, max=199.7514692043265, mean=50.84, nulos=0
avg_purchase_value: min=118.93767633551786, max=190.00896040297204, mean=152.19, nulos=0
purchase_frequency: min=4.733333333333333, max=72.0, mean=18.12, nulos=0
avg_discount_used: min=0.0, max=14.96257824333469, mean=0.29, nulos=0
online_purchases: min=0, max=12, mean=2.14, nulos=0
in_store_purchases: min=0, max=11, mean=0.96, nulos=0
total_sales: min=0.04, max=990.1493097576405, mean=152.19, nulos=0
total_transactions: min=5, max=51, mean=29.61, nulos=0
total_items_purchased: min=10, max=109, mean=58.21, nulos=0
promotion_effectiveness: min=0.4166666666666667, max=0.75, mean=0.60, nulos=0
days_since_last_purchase: min=414, max=1273, mean=844.02, nulos=0


In [723]:
# Dataset final con todas las columnas procesadas
df_final = df_clean
df_final = df_final.reset_index(drop=True)
df_final.to_csv("dataset_final_all_columns.csv", index=False)

## Dataset Data Analyst

In [724]:
# Dataset resumido por cliente

df_agg = df_final.groupby("customer_id").agg({
    # Numéricas: agregaciones típicas
    "age": "first",
    "membership_years": "first",
    "number_of_children": "first",
    "quantity": "mean",
    "unit_price": "mean",
    "avg_purchase_value": "mean",
    "purchase_frequency": "mean",
    "avg_discount_used": "mean",
    "online_purchases": "sum",
    "in_store_purchases": "sum",
    "total_sales": "sum",
    "total_transactions": "sum",
    "total_items_purchased": "sum",
    "promotion_effectiveness": "mean",
    "days_since_last_purchase": "min",

    # Categóricas: valor más frecuente
    "gender": lambda x: x.mode()[0],
    "income_bracket": lambda x: x.mode()[0],
    "marital_status": lambda x: x.mode()[0],
    "education_level": lambda x: x.mode()[0],
    "occupation": lambda x: x.mode()[0],
    "product_category": lambda x: x.mode()[0],
    "promotion_type": lambda x: x.mode()[0],


    # Fechas
    "transaction_date": "max",
    "last_purchase_date": "max",

    # Flags binarios
    "loyalty_program": "max",
    "churn": "max",
    "promo_flag": "max"

}).reset_index()


# Exportar CSV para Data Analyst

df_agg.to_csv("dataset_analyst_by_customer.csv", index=False)


**Documentación para Data analyst:**


* Diccionario de columnas con descripción y tipo.

* Rango y estadística clave de cada variable.

* Notas sobre imputaciones, clipping y derivadas.

## Dataset Data Scientist

### Codificación de variables categóricas: One-hot encoding

In [725]:
categorical_cols = [
    "gender", "income_bracket", "marital_status",
    "education_level", "occupation",
    "product_category", "promotion_type"
]

df_final_encoded = pd.get_dummies(df_final, columns=categorical_cols, drop_first=False)

In [726]:
# Dataset final para Data Scientist
df_final_encoded.to_csv("customer_dataset_for_ml.csv", index=False)

**Documentación para Data Scientist:**


* Diccionario de columnas codificadas (qué representa cada columna 0/1).

* Notas sobre limpieza y derivadas.

* Estadísticas resumidas de features numéricas.

## Validación final

### 1️⃣ Chequeo rápido de nulos / NaN

In [727]:
def check_nulls(df, name="dataset"):
    print(f"\n📌 Null check → {name}")
    nulls = df.isna().sum()
    nulls = nulls[nulls > 0]

    if nulls.empty:
        print("✅ No hay valores nulos ni NaN")
    else:
        print("❌ Columnas con nulos:")
        print(nulls)


In [728]:
check_nulls(df_final, "df_final")
check_nulls(df_agg, "df_agg")



📌 Null check → df_final
✅ No hay valores nulos ni NaN

📌 Null check → df_agg
✅ No hay valores nulos ni NaN


### 2️⃣Verificar columnas binarias (0/1)

In [729]:
def check_binary_cols(df, binary_cols):
    print("\n📌 Binary columns check")
    for col in binary_cols:
        if col in df.columns:
            invalid = df[~df[col].isin([0,1]) & df[col].notna()]
            if len(invalid) == 0:
                print(f"✅ {col}: OK (solo 0/1)")
            else:
                print(f"❌ {col}: valores inválidos →")
                print(df[col].value_counts())


In [730]:
binary_cols = ["loyalty_program", "churn", "promo_flag" ]
check_binary_cols(df_final, binary_cols)



📌 Binary columns check
✅ loyalty_program: OK (solo 0/1)
✅ churn: OK (solo 0/1)
✅ promo_flag: OK (solo 0/1)


### 3️⃣ Verificación de rangos numéricos

In [731]:
def check_numeric_ranges(df, num_cols):
    print("\n📌 Numeric ranges check")
    for col in num_cols:
        if col in df.columns:
            min_v = df[col].min()
            max_v = df[col].max()
            if min_v < 0:
                print(f"⚠️ {col}: valor negativo detectado (min={min_v})")
            else:
                print(f"✅ {col}: min={min_v}, max={max_v}")


In [732]:
check_numeric_ranges(df_final, num_cols)



📌 Numeric ranges check
✅ age: min=18, max=70
✅ membership_years: min=1, max=15
✅ number_of_children: min=0, max=4
✅ quantity: min=1, max=12
✅ unit_price: min=0.02, max=199.7514692043265
✅ avg_purchase_value: min=118.93767633551786, max=190.00896040297204
✅ purchase_frequency: min=4.733333333333333, max=72.0
✅ avg_discount_used: min=0.0, max=14.96257824333469
✅ online_purchases: min=0, max=12
✅ in_store_purchases: min=0, max=11
✅ total_sales: min=0.04, max=990.1493097576405
✅ total_transactions: min=5, max=51
✅ total_items_purchased: min=10, max=109
✅ promotion_effectiveness: min=0.4166666666666667, max=0.75
✅ days_since_last_purchase: min=414, max=1273


### 4️⃣ Chequeo de fechas (consistencia temporal)

In [733]:
def check_dates(df, date_cols):
    print("\n📌 Date columns check")
    for col in date_cols:
        if col in df.columns:
            if not pd.api.types.is_datetime64_any_dtype(df[col]):
                print(f"❌ {col}: no es datetime")
            else:
                print(f"✅ {col}: OK ({df[col].min()} → {df[col].max()})")


In [734]:
check_dates(df_final, date_cols)



📌 Date columns check
✅ transaction_date: OK (2022-07-05 00:00:00 → 2024-11-10 00:00:00)
✅ last_purchase_date: OK (2022-07-05 00:00:00 → 2024-11-10 00:00:00)


### 5️⃣ Valores categóricos raros (cardinalidad)

In [735]:
def check_categorical_cols(df, cat_cols, max_unique=20):
    print("\n📌 Categorical consistency check")
    for col in cat_cols:
        if col in df.columns:
            uniq = df[col].nunique(dropna=False)
            print(f"🔹 {col}: {uniq} valores únicos")
            if uniq <= max_unique:
                print(df[col].value_counts())


In [736]:
check_categorical_cols(df_agg, categorical_cols)



📌 Categorical consistency check
🔹 gender: 3 valores únicos
gender
Other     169
Male      169
Female    162
Name: count, dtype: int64
🔹 income_bracket: 3 valores únicos
income_bracket
High      192
Low       164
Medium    144
Name: count, dtype: int64
🔹 marital_status: 3 valores únicos
marital_status
Single      184
Divorced    158
Married     158
Name: count, dtype: int64
🔹 education_level: 4 valores únicos
education_level
Master's       137
PhD            127
Bachelor's     123
High School    113
Name: count, dtype: int64
🔹 occupation: 5 valores únicos
occupation
Employed         109
Self-Employed    108
Unemployed       101
Retired           96
Student           86
Name: count, dtype: int64
🔹 product_category: 6 valores únicos
product_category
Beauty         95
Books          93
Electronics    88
Clothing       84
Sports         75
Home           65
Name: count, dtype: int64
🔹 promotion_type: 3 valores únicos
promotion_type
no_promotion    373
BOGO             67
Discount         6

### 6️⃣ Chequeo de duplicados

In [737]:
def check_duplicates(df, subset=None, name="dataset"):
    print(f"\n📌 Duplicate check → {name}")
    dups = df.duplicated(subset=subset).sum()
    if dups == 0:
        print("✅ No hay duplicados")
    else:
        print(f"❌ {dups} duplicados encontrados")


In [738]:
check_duplicates(df_agg, subset=["customer_id"], name="df_agg")



📌 Duplicate check → df_agg
✅ No hay duplicados


### 7️⃣ Check FINAL tipo “semáforo”

In [739]:
def final_dataset_check(df, name):
    print(f"\n🚦 FINAL CHECK → {name}")
    print("Filas:", df.shape[0])
    print("Columnas:", df.shape[1])
    print("NaN totales:", df.isna().sum().sum())


In [740]:
final_dataset_check(df_final, "Dataset Final (All Columns)")
final_dataset_check(df_agg, "Dataset Analyst by Customer")



🚦 FINAL CHECK → Dataset Final (All Columns)
Filas: 35844
Columnas: 31
NaN totales: 0

🚦 FINAL CHECK → Dataset Analyst by Customer
Filas: 500
Columnas: 28
NaN totales: 0
